# Run MPNN and then submit to AF2 using Google Colab
The purpose of this notebook is to run MPNN and AF2 (after you have run RFDiffusion, if that's what you are doing). This runs both MPNN and AF2 serially.

---
### AWS settings:
Image: Data Science 3.0
Instance: ml.g4dn.xlarge

First install the jax version of colabdesign

In [6]:
%%bash
pip install -U "jax[cuda11_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
pip install git+https://github.com/sokrypton/ColabDesign.git
mkdir params
wget -qnc  https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar
tar -xf alphafold_params_2022-12-06.tar -C params

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Obtaining dependency information for jax[cuda11_pip] from https://files.pythonhosted.org/packages/b5/5b/5131520dd9a384a640399e5efe4324fdee9e8a48685a33d08eb47140ccc3/jax-0.4.18-py3-none-any.whl.metadata
  Using cached jax-0.4.18-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for ml-dtypes>=0.2.0 from https://files.pythonhosted.org/packages/90/31/ec94e33a799323a8c37d1883f44b517c38d9defa7667db97cba212384d71/ml_dtypes-0.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached ml_dtypes-0.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.18%2Bcuda11.cudnn86-cp310-cp310-manylinux2014_x86_64.whl (119.7 MB)
  Using cached nvidia_cublas_cu11-11.11.3.6-py3-none-manylinux1_x86_64.whl (417.9 MB)
  Us

  Cloning https://github.com/sokrypton/ColabDesign.git to /tmp/pip-req-build-ivyjhfle


  Running command git clone --filter=blob:none --quiet https://github.com/sokrypton/ColabDesign.git /tmp/pip-req-build-ivyjhfle


  Resolved https://github.com/sokrypton/ColabDesign.git to commit 86be826a2879bee1b7deb0d243ccfb7f6040975b
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for py3Dmol from https://files.pythonhosted.org/packages/b5/3d/052e5932ef95624e118b886feb58a9c60595e89da74515604933b6b0e6a5/py3Dmol-2.0.4-py2.py3-none-any.whl.metadata
  Using cached py3Dmol-2.0.4-py2.py3-none-any.whl.metadata (2.1 kB)
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached biopython-1.81-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Obtaining dependency information for chex from https://files.pythonhosted.org/packages/8b/c5/ab99c61d1384f89fe0d89b4b105c1ad22dab98cfe8c78136fb8c3f75f75b/che

mkdir: cannot create directory ‘params’: File exists
tar: params_model_1.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_2.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_3.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_4.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_5.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_1_ptm.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_2_ptm.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_3_ptm.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_4_ptm.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: params_model_5_ptm.npz: Cannot change ownership to uid 995210, gid 89939: Invalid argument
tar: pa

CalledProcessError: Command 'b'pip install -U "jax[cuda11_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html\npip install git+https://github.com/sokrypton/ColabDesign.git\nmkdir params\nwget -qnc  https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar\ntar -xf alphafold_params_2022-12-06.tar -C params\n'' returned non-zero exit status 2.

---
### Create folder where you want to save your MPNN and AF2 results
*N.B. it's a good practice to create and save new folders for every design task you are running! I like to also save everything with the date so I know when things were run :)*

Edit the cell below:

In [7]:
# Write here to point to where you wsnt your AF2 results to go
!mkdir -p /root/data/AF2_results/il7_binders/

# Write here
contig="80-80:B1-196"
rfpdb="/root/AI4PD_2023/notebooks/AI4PD_tutorials/data/results/rfdiffusion/binder/il7_0.pdb" # path to a single PDB
output="/root/data/AF2_results/il7_binders/" # Same path as above

---
## Run designability test on a single PDB

There are some important flags you will want to know.

**Required Arguments:**
- `pdb` is the path to your pdb you want to test the designability for
- `loc` is where you want to save your MPNN sequences and your AF2 results
- `contigs` are your contigs you used with RFDiffusion. This is set up to automatically detect which sequences you want to fix, etc.

**Optional Arguments:**
- `num_designs` is number of designs to evaluate (design and fold). This works well if you have been designing multiple sequences with RFDiffusion you want to design in the same way.
- `mpnn_sampling_temp` is the sampling temperature for ProteinMPNN
- `num_seqs` is how many sequences per backbone you want to design with ProteinMPNN
- `use_soluble` uses the soluble weights for ProteinMPNN
- `intial_guess` used an initial guess for AlphaFold2 from the input PDB

In [ ]:
# Edit here
!python3 /opt/conda/lib/python3.10/site-packages/colabdesign/rf/designability_test.py 
--pdb={rfpdb} \
--loc={output} \
--contigs={contig} \
--initial_guess \
--num_seqs=8 

If you are happy with the designability of your designs, you can then go ahead and run AF2 for all your designs with the `num_designs` flag! Your command instead will look like the following:

In [ ]:
# Edit here
num_designs=10 # how ever many RFDiffusion outputs you have

!python3 /opt/conda/lib/python3.10/site-packages/colabdesign/rf/designability_test.py 
--pdb={rfpdb} \
--num_designs={num_designs} \
--loc={output} \
--contigs={contig} \
--initial_guess \
--num_seqs=8 

---
## Visualize our designs
The following cells are for us to take a look at our designs once we are done with running AF2

In [11]:
import os
import py3Dmol

def extract_structures_from_dir(directory):
    pdb_files = [f for f in os.listdir(directory) if f.endswith('.pdb')]
    structures = []
    for pdb_file in pdb_files:
        with open(os.path.join(directory, pdb_file), 'r') as file:
            structures.append(file.read())
    return structures

In [12]:
## WRITE HERE
directory_to_pdbs = "/root/data/AF2_results/il7_binders/all_pdb"

structures = extract_structures_from_dir(directory_to_pdbs)  # replace with your directory

# Display each structure in a separate window
for structure in structures:
    view = py3Dmol.view()
    view.addModel(structure, format="pdb")
    view.setStyle({"chain": "B"}, {"cartoon": {"color": "blue", "opacity": 0.6}})
    view.setStyle({"chain": "A"}, {"cartoon": {"color": "green"}})
    view.zoomTo()
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol